# Writing Data
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

It is possible to write out the data at any point in a Dataflow. These writes are added as steps to the resulting Dataflow and will be executed every time the Dataflow is executed. Since there are no limitations to how many write steps there are in a pipeline, this makes it easy to write out intermediate results for troubleshooting or to be picked up by other pipelines.

It is important to note that the execution of each write results in a full pull of the data in the Dataflow. For example, a Dataflow with three write steps will read and process every record in the dataset three times.

In [1]:
import azureml.dataprep as dprep

## Writing to Files

Data can be written to files in any of our supported locations (Local File System, Azure Blob Storage, and Azure Data Lake Storage). In order to parallelize the write, the data is written to multiple partition files. A sentinel file named SUCCESS is also output once the write has completed. This makes it possible to identify when an intermediate write has completed without having to wait for the whole pipeline to complete.

> When running a Dataflow in Spark, attempting to execute a write to an existing folder will fail. It is important to ensure the folder is empty or use a different target location per execution.

The following file formats are currently supported:
- Delimited Files (CSV, TSV, etc.)
- Parquet Files

We'll start by loading data into a Dataflow which will be re-used with different formats.

In [2]:
dflow = dprep.auto_read_file('../data/crime.txt')
dflow = dflow.to_number('Column2')
dflow.head(5)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,10140490.0,"azureml.dataprep.native.DataPrepError(""'Micros...",329907.0,7/5/2015,23:50,50.0,XX,N,NEWLAND AVE 820 THEFT
1,10139776.0,"azureml.dataprep.native.DataPrepError(""'Micros...",329265.0,7/5/2015,23:30,11.0,XX,W,MORSE AVE 460 BATTERY
2,10140270.0,"azureml.dataprep.native.DataPrepError(""'Micros...",329253.0,7/5/2015,23:20,121.0,XX,S,FRONT AVE 486 BATTERY
3,10139885.0,"azureml.dataprep.native.DataPrepError(""'Micros...",329308.0,7/5/2015,23:19,51.0,XX,W,DIVISION ST 610 BURGLARY
4,10140379.0,"azureml.dataprep.native.DataPrepError(""'Micros...",329556.0,7/5/2015,23:00,12.0,XX,W,LAKE ST 930 MOTOR VEHICLE THEFT


### Delimited Files

Here we create a dataflow with a write step.

This operation is lazy until we invoke `run_local` (or any operation that forces execution like `to_pandas_dataframe`), only then will we execute the write operation.

In [3]:
dflow_write = dflow.write_to_csv(directory_path=dprep.LocalFileOutput('./test_out/'))

dflow_write.run_local()

dflow_written_files = dprep.read_csv('./test_out/part-*')
dflow_written_files.head(5)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,10140490,ERROR,329907,7/5/2015,23:50,50,XX,N,NEWLAND AVE 820 THEFT
1,10139776,ERROR,329265,7/5/2015,23:30,11,XX,W,MORSE AVE 460 BATTERY
2,10140270,ERROR,329253,7/5/2015,23:20,121,XX,S,FRONT AVE 486 BATTERY
3,10139885,ERROR,329308,7/5/2015,23:19,51,XX,W,DIVISION ST 610 BURGLARY
4,10140379,ERROR,329556,7/5/2015,23:00,12,XX,W,LAKE ST 930 MOTOR VEHICLE THEFT


The data we wrote out contains several errors in the numeric columns due to numbers that we were unable to parse. When written out to CSV, these are replaced with the string "ERROR" by default. We can parameterize this as part of our write call. In the same vein, it is also possible to set what string to use to represent null values.

In [4]:
dflow_write_errors = dflow.write_to_csv(directory_path=dprep.LocalFileOutput('./test_out/'), 
                                        error='BadData',
                                        na='NA')
dflow_write_errors.run_local()
dflow_written = dprep.read_csv('./test_out/part-*')
dflow_written.head(5)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,10140490,BadData,329907,7/5/2015,23:50,50,XX,N,NEWLAND AVE 820 THEFT
1,10139776,BadData,329265,7/5/2015,23:30,11,XX,W,MORSE AVE 460 BATTERY
2,10140270,BadData,329253,7/5/2015,23:20,121,XX,S,FRONT AVE 486 BATTERY
3,10139885,BadData,329308,7/5/2015,23:19,51,XX,W,DIVISION ST 610 BURGLARY
4,10140379,BadData,329556,7/5/2015,23:00,12,XX,W,LAKE ST 930 MOTOR VEHICLE THEFT


### Parquet Files

Similar to `write_to_csv`, `write_to_parquet` returns a new Dataflow with a Write Parquet Step which hasn't been executed yet.

Then we run the Dataflow with `run_local`, which executes the write operation.

In [5]:
dflow_write_parquet = dflow.write_to_parquet(directory_path=dprep.LocalFileOutput('./test_parquet_out/'),
                                             error='MiscreantData')

dflow_write_parquet.run_local()

dflow_written_parquet = dprep.read_parquet_file('./test_parquet_out/part-*')
dflow_written_parquet.head(5)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,10140490.0,MiscreantData,329907.0,7/5/2015,23:50,50.0,XX,N,NEWLAND AVE 820 THEFT
1,10139776.0,MiscreantData,329265.0,7/5/2015,23:30,11.0,XX,W,MORSE AVE 460 BATTERY
2,10140270.0,MiscreantData,329253.0,7/5/2015,23:20,121.0,XX,S,FRONT AVE 486 BATTERY
3,10139885.0,MiscreantData,329308.0,7/5/2015,23:19,51.0,XX,W,DIVISION ST 610 BURGLARY
4,10140379.0,MiscreantData,329556.0,7/5/2015,23:00,12.0,XX,W,LAKE ST 930 MOTOR VEHICLE THEFT
